# Executive Summary

Abc network intends to launch a new docu-series on property flipping called "house hunters". The show's target audience is Americans who are passionate about transforming homes and reselling them at a profit. Its unique selling point is taking a data driven approach to property flipping. The first season of the series will feature houses in Ames, Iowa. I have been hired as a data scientist to  then idenfity features that are the best predictors of sale prices.

By analysing the housing dataset from Ames, Iowa Assessor’s Office containing individual property sales from 2006 - 2010, I have created a predictive model using linear regression with Ridge regularization.  A baseline model describing the average sale price of the entire dataset was used as a benchmark to beat when building the model. To verify that our model works well, we have been provided with a test sample with similar features to the dataset to test on kaggle.

This analyses is split in two parts. The first part is for exploratory data anlysis and the second part for model construction and regularization. *This notebook covers the first part on EDA.*

The model includes a total of 14 features: 
- 4 continuous
- 3 discrete
- 3 nominal
- 2 ordinal
- 2 polynomial

## Problem Statement

The primary stakeholders of this analyses is the team in charge of "house hunters" at abc and the secondary stakeholders are American TV viewers who are keen on property flipping. Hence the analyses aims to deconstruct the dataset into digestible information and reduce the features to handful which most strongly predict sale prices, while uncovering interesting relationships between features and sale prices which would help in developing strategies for property flipping.


In [ ]:
# Importing the relevant libraries
import numpy as np
import pandas as pd
from numpy import linspace

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import PolynomialFeatures

# Contents

- [Data Cleaning and Processing](#cleaning)
    - [Addressing Null Values](#null)
    - [Outliers](#outliers)

- [Exploratory Data Analysis](#eda)
    - [Continuous Features](#cont)
    - [Discrete Features](#disc)
    - [Ordinal Features](#ordi)
    - [Nominal Features](#nomi)
    - [Polynomial Features](#poly)
    - [Target Feature](#target)
- [Selected Features](#select)

# Data Cleaning and Processing
<a id='cleaning'></a>

In [ ]:
#import train dataset
train_df = pd.read_csv('./datasets/train.csv')

In [ ]:
#General function for checking dataframe
def df_checks (df):
    print('Dataframe dimensions: \n' + str(df.shape) + '\n')
    print('Column Names:')
    print(df.columns)
    print('\n')
    print('Missing Values: \n' + str(df.isnull().sum().sort_values(ascending = False).head(10)) + '\n')
    print('Statistics: \n' + str(df.describe()) + '\n')
    print('Datatypes: \n' + str(df.dtypes) + '\n')
    print('Information: \n')
    print(df.info())

In [ ]:
df_checks(train_df)

In [ ]:
#Create new dataframe to store clean data
train_clean = train_df

In [ ]:
#make changes to column names for convenience

train_clean.columns = train_clean.columns.map(lambda x: (str(x.replace(' ','_'))).lower())
train_clean.columns = train_clean.columns.map(lambda x: (str(x.replace('/','_'))).lower())

## Summary of train dataset
Total of 81 features and 2051 rows in the dataset. The tables below summarize the features by their types along with their description and missing values.

### Target feature: 1

|No|Column name|Data type|Description|Null values|
|:---:|:---|:---:|---|---:|
|1|saleprice|int|Sale price of property in $|0|

### Continuous features: 19

|No|Column name|Data type|Description|Null values|
|:---:|:---|:---:|---|---:|
|1|lot_frontage|float|Linear feet of street connected to property|330|
|2|lot_area|int|Lot size in square feet|0|
|3|mas_vnr_area|float|Masonry veneer area in square feet|22|
|4|basmtfin_sf_1|float|Type 1 basement area in square feet|1|
|5|basmtfin_sf_2|float|Type 2 basement area in square feet|1|
|6|bsmt_unf_sf|float|Unfinished basement area in square feet|1|
|7|total_bsmt_sf|float|Total square feet of basement area|1|
|8|1st_flr_sf|int|First floor area in square feet|0|
|9|2nd_flr_sf|int|Second floor area in square feet|0|
|10|low_qual_fin_sf|int|Low quality finished in square feet (all floors)|0|
|11|gr_liv_area|int|Above grade (ground) living area square feet|0|
|12|garage_area|float|Size of garage in square feet|1|
|13|wood_deck_sf|int|Wood deck area in square feet|0|
|14|open_porch_sf|int|Open porch area in square feet|0|
|15|enclosed_porch|int|Enclosed porch area in square feet|0|
|16|3ssn_porch|int|Three season porch area in square feet|0|
|17|screen_porch|int|Screen porch area in square feet|0|
|18|pool_area|int|Pool area in square feet|0|
|19|misc_val|int|Value of miscellaneous feature in $|0|


### Discrete features: 14

|No|Column name|Data type|Description|Null values|
|:---:|:---|:---:|---|---:|
|1|year_built|int|Original construction date|0|
|2|year_remod_add|int|Remodel date (same as construction date if no remodeling or additions)|0|
|3|bsmt_full_bath|float|Basement full bathrooms|2|
|4|bsmt_half_bath|float|Basement half bathrooms|2|
|5|full_bath|int|Full bathrooms above grade|0|
|6|half_bath|int|Half baths above grade|0|
|7|bedroom_abvgr|int|Bedrooms above grade (does NOT include basement bedrooms)|0|
|8|kitchen_abvgr|int|Kitchens above grade|0|
|9|totrms_abvgrd|int|Total rooms above grade (does not include bathrooms)|0|
|10|fireplaces|int|Number of fireplaces|0|
|11|garage_yr_blt|float|Year garage was built|114|
|12|garage_cars|float|Garage car capacity|1|
|13|mo_sold|int|Month Sold (MM)|0|
|14|yr_sold|int|Year Sold (YYYY)|0|

### Nominal features: 24

|No|Column name|Data type|Description|Null values|
|:---:|:---|:---:|---|---:|
|1|ms_subclass|int|Identifies the type of dwelling involved in the sale|0|
|2|ms_zoning|str|Identifies the general zoning classification of the sale.|0|
|3|street|str|Type of road access to property|0|
|4|alley|str|Type of alley access to property|1911|
|5|land_contour|str|Flatness of the property|0|
|6|lot_config|str|Lot configuration|0|
|7|neighborhood|str|Physical locations within Ames city limits (map available)|0|
|8|condition_1|str|Proximity to various conditions|0|
|9|condition_2|str|Proximity to various conditions (if more than one is present)|0|
|10|bldg_type|str|Type of dwelling|0|
|11|house_style|str|Style of dwelling|0|
|12|roof_style|str|Type of roof|0|
|13|root_matl|str|Roof material|0|
|14|exterior_1st|str|Exterior covering on house|0|
|15|exterior_2nd|str|Exterior covering on house (if more than one material)|0|
|16|mas_vnr_type|str|Masonry veneer type|22|
|17|foundation|str|Type of foundation|0|
|18|heating|str|Type of heating|0|
|19|central_air|str|Central air conditioning|0|
|20|garage_type|str|Garage location|113|
|21|misc_feature|str|Miscellaneous feature not covered in other categories|1986|
|22|sale_type|str|Type of sale|0|
|23|pid|int|parcel indentification number|0|
|24|id|int|unique identification number|0|

### Ordinal features: 23

|No|Column name|Data type|Description|Null values|
|:---:|:---|:---:|---|---:|
|1|lot_shape|str|General shape of property|0|
|2|utilities|str|Type of utilities available|0|
|3|land_slope|str|Slope of property|0|
|4|overall_qual|int|Rating for overall material and finish of the house|0|
|5|overall_cond|int|Ratiing for overall condition of the house|0|
|6|exter_qual|str|Evaluates the quality of the material on the exterior|0|
|7|exter_cond|str|Evaluates the present condition of the material on the exterior|0|
|8|bsmt_qual|str|Evaluates height of basement|55|
|9|bsmt_cond|str|Evaluates the general condition of the basement|55|
|10|bsmt_exposure|str|Refers to walkout or garden level walls|58|
|11|bsmtfin_type_1|str|Rating of basement finished area|55|
|12|bsmtfin_type_2|str|Rating of basement finished area (if multiple types)|56|
|13|heatingqc|str|Heating quality and condition|0|
|14|electrical|str|Electrical system|0|
|15|kitchen_qual|str|Kitchen quality|0|
|16|functional|str|Home functionality (Assume typical unless deductions are warranted)|0|
|17|fireplace_qu|str|Fireplace quality|1000|
|18|garage_finish|str| Interior finish of the garage|114|
|19|garage_qual|str|Garage quality|114|
|20|garage_cond|str|Garage condition|114|
|21|paved_drive|str|Paved driveway|0|
|22|pool_qc|str|Pool quality|2042|
|23|fence|str|Fence quality|1651|

<a id='null'></a>
## Addressing null values

There are a total of 26 features with null values

In [ ]:
train_clean.isnull().sum().sort_values(ascending = False).head(27)

Continuous Variables: 7

|Column|Null values|
|---|---|
|lot_frontage|330|
|mas_vnr_area|22|
|total_bsmt_sf|1|
|bsmt_unf_sf|1|
|bsmtfin_sf_2|1|
|bsmtfin_sf_1|1|
|garage_area|1|

Discrete Variables: 4

|Column|Null values|
|---|---|
|garage_yr_blt|114|
|bsmt_half_bath|2|
|bsmt_full_bath|2|
|garage_cars|1|


Nominal Variables: 4

|Column|Null values|
|---|---|
|misc_feature|1986|
|alley|1911|
|garage_type|113|
|mas_vnr_type|22|



Ordinal Varibales: 11

|Column|Null values|
|---|---|
|pool_qc|2042|
|fence|1651|
|fireplace_qu|1000|
|garage_finish|114|
|garage_qual|114|
|garage_cond|114|
|bsmt_exposure|58|
|bsmtfin_type_2|56|
|bsmtfin_type_1|55|
|bsmt_cond|55|
|bsmt_qual|55|

### Garage Data
There is a similar number of null values for garage data. This leads to my hypothesis that the missing garage data is due to house having no garage

In [ ]:
missing_garage = train_clean[train_clean['garage_finish'].isnull() == True]
missing_garage[['garage_area','garage_yr_blt','garage_cars','garage_type','garage_qual','garage_cond']].head(20)

Since the `garage_area` and `garage_cars` are `0` for rows with null values for the other garage features, we can safely assume that my hypothesis is true and edit the dataset by replacing the null values with `None` or `0`. For `garage_yr_blt`, it would be safe to impude the year which the house was built.

In [ ]:
train_clean['garage_finish'] = train_clean['garage_finish'].fillna('None')
train_clean['garage_area'] = train_clean['garage_area'].fillna(0)
train_clean['garage_yr_blt'] = train_clean['garage_yr_blt'].fillna(train_clean['year_built'])
train_clean['garage_cars'] = train_clean['garage_cars'].fillna(0)
train_clean['garage_type'] = train_clean['garage_type'].fillna('None')
train_clean['garage_qual'] = train_clean['garage_qual'].fillna('None')
train_clean['garage_cond'] = train_clean['garage_cond'].fillna('None')

### Basement Data
I form a similar hypothesis for null values in basement features.

In [ ]:
missing_basement = train_clean[train_clean['bsmt_exposure'].isnull() == True]
missing_basement[['total_bsmt_sf','bsmt_unf_sf','bsmtfin_sf_2','bsmtfin_sf_1','bsmt_half_bath','bsmt_full_bath','bsmtfin_type_2','bsmtfin_type_1','bsmt_cond','bsmt_qual']].head(20)

We can safely assume that the null values for basement features represent the house having no basement, inluding `bsmt_full_bath` and `bsmt_half_bath`.

In [ ]:
train_clean['bsmt_exposure'] = train_clean['bsmt_exposure'].fillna('None')
train_clean['total_bsmt_sf'] = train_clean['total_bsmt_sf'].fillna(0)
train_clean['bsmt_unf_sf'] = train_clean['bsmt_unf_sf'].fillna(0)
train_clean['bsmtfin_sf_2'] = train_clean['bsmtfin_sf_2'].fillna(0)
train_clean['bsmtfin_sf_1'] = train_clean['bsmtfin_sf_1'].fillna(0)
train_clean['bsmt_half_bath'] = train_clean['bsmt_half_bath'].fillna(0)
train_clean['bsmt_full_bath'] = train_clean['bsmt_full_bath'].fillna(0)
train_clean['bsmtfin_type_2'] = train_clean['bsmtfin_type_2'].fillna('None')
train_clean['bsmtfin_type_1'] = train_clean['bsmtfin_type_1'].fillna('None')
train_clean['bsmt_cond'] = train_clean['bsmt_cond'].fillna('None')
train_clean['bsmt_qual'] = train_clean['bsmt_qual'].fillna('None')

In [ ]:
train_clean.isnull().sum().sort_values(ascending = False).head(10)

### Other features with null values
There are still 8 features remaining containing null values. These will be dealt with individually.

#### Pools
`pool_qc` is an ordinal feature with values representing the quality of the pool. Its possible values are a rating system (no pool, fair, typical, good, excellent)

In [ ]:
train_clean['pool_qc'].unique()

In [ ]:
missing_pool = train_clean[train_clean['pool_qc'].isnull() == True]
missing_pool[['pool_area','pool_qc']].head(10)

We can see that the null values represent the house having no pool. Also, the unique values in the dataset exclude the "no pool" rating. Therefore it would be safe to replace the null values with `None` for consistency

In [ ]:
train_clean['pool_qc'] = train_clean['pool_qc'].fillna('None')

#### Misc Features
misc_feature is a nominal feature which represents other features that are not covered by others. In this particular dataset, the misc features includes: elevator, 2nd garage, other, shed, tennis court, none.

In [ ]:
train_clean['misc_feature'].unique()

We can safely assume that the null values here represent the house having no additional features. It would be safe to replace the null values with `None`.

In [ ]:
train_clean['misc_feature'] = train_clean['misc_feature'].fillna('None')

#### Alleys
`alley` is a nominal feature which dictates the type of alley access to the property. The feature includes values which represent: gravel, paved, or no alley.

In [ ]:
train_clean['alley'].unique()

We can safely assume that the null values here represent the house having no alley access. It would be safe to replace the null values with `None`.

In [ ]:
train_clean['alley'] = train_clean['alley'].fillna('None')

#### Fences
`fence` is an ordinal feature which represents the quality of the fence. The feature describes the fences of the property as having: good privacy, minimum privacy, good wood,  minimum wood/wire, or no fence.

In [ ]:
train_clean['fence'].unique()

We can safely assume that the null values here represent the house having no fence. It would be safe to replace the null values with `None`.

In [ ]:
train_clean['fence'] = train_clean['fence'].fillna('None')

#### Fireplaces
`fireplace_qu` is an ordinal variable representing the fireplace quality. The feature includes a rating system in ascending order: no fireplace, poor, fair, average, good and excellent.

In [ ]:
train_clean['fireplace_qu'].unique()

In [ ]:
missing_fire = train_clean[train_clean['fireplace_qu'].isnull() == True]
missing_fire[['fireplaces','fireplace_qu']].head(10)

Checking `fireplace_qu` against the number of fireplaces confirms that the null values represent missing fireplace. We can safely replace the null values with `None`.

In [ ]:
train_clean['fireplace_qu'] = train_clean['fireplace_qu'].fillna('None')

#### Masonry
The features `mas_vnr_type` and `mas_vnr_area` represent the masonry veneer type and area respectively. The former is a nominal variable taking up different values representing the material used in its construction and the latter represents its size.

In [ ]:
train_clean['mas_vnr_type'].unique()

In [ ]:
missing_mas = train_clean[train_clean['mas_vnr_type'].isnull() == True]
missing_mas[['mas_vnr_type','mas_vnr_area']]

Since the null values for `mas_vnr_type` and `mas_vnr_type` coincide, we can assume that the null represents an absence of masonry veneer.

In [ ]:
train_clean['mas_vnr_type'] = train_clean['mas_vnr_type'].fillna('None')
train_clean['mas_vnr_area'] = train_clean['mas_vnr_area'].fillna(0)

#### Lot Frontage
`lot_frontage` is a continuous feature which refers to the distance between street to property in feet. We must be careful when dealing with the null values for this feature as we do not want the imputation to affect the train/test split. The number of null values is significant here (330) therefore simply replacing them with `0` may not be an accurate representation of the missing values.

For EDA purposes I will temporarily fill these values appropriately. If the feature is selected for the model, the changes will be performed again based only on the train dataset.

A quick way to resolve this issue is to replace the null values with the overall mean `lot_frontage` of the rest of the dataset. However, it would be more accurate to replace the null values with the mean `lot_frontage` of their respective neighborhood.

In [ ]:
plt.figure(figsize = (15,10))
sns.boxplot(y= train_clean['neighborhood'], x= train_clean['lot_frontage']);

In [ ]:
train_clean.groupby('neighborhood')['lot_frontage'].mean()

There is not additional information on the neighborhoods `GrnHill` and `Landmrk` on `lot_frontage`. We will replace those with the overall mean lot_frontage before impudation instead.

In [ ]:
overall_mean_frontage = train_clean['lot_frontage'].mean()
train_clean['lot_frontage'] = train_clean.groupby('neighborhood')['lot_frontage'].transform(lambda x: x.fillna(x.mean()))
train_clean['lot_frontage'] = train_clean['lot_frontage'].fillna(overall_mean_frontage)

In [ ]:
train_clean.isnull().sum().sum()

Now we have no more null values!

## Checking Data types

The features can be categorised into continuous, discrete, nominal and ordinal. We expect continuous and discrete features to be integer or float datatypes.
Nominal features should be object or category datatype
Ordinal features can be integer/float or object/category datatypes as it depends on the ranking method.

In [ ]:
train_clean.info()

The only feature which does not match its datatype as per category is `ms_subclass`. It is an integer when it is a nominal feature. This must be changed to a categorical datatype.

In [ ]:
#Changing dtype for ms_subclass
train_clean['ms_subclass'] = train_clean['ms_subclass'].astype(dtype = 'category')

<a id='outliers'></a>
## Outliers

One final step before we proceed to EDA is dealing with outlier data. In the documentation for the Ames Housing Dataset, it is stated under SPECIAL NOTES that there are some unusual sales which turn out to be outliers. It is also recommended that these outliers are removed before modeling. We can visualize the outliers by plotting `gr_liv_area` vs `saleprice`, then plotting a vertical line to distinguish houses over 4000 sq ft.

In [ ]:
plt.figure(figsize = (15,15))
sns.scatterplot(data = train_clean, x = 'gr_liv_area', y = 'saleprice');
plt.axvline(4000, 0,600000, ls = '--', lw = 1, c = 'black');

In [ ]:
#Locating the outliers based on living area
train_clean[(train_clean['gr_liv_area'] >= 4000)]

Additionally, two other outliers were identified during the modeling process. The model over predicted the sale prices of these two houses

In [ ]:
out_1 = pd.DataFrame(train_clean.iloc[125,:])
out_2 = pd.DataFrame(train_clean.iloc[348,:])
outliers = pd.concat([out_1,out_2], axis = 1)
outliers = outliers.T
outliers

In [ ]:
outliers['overall_qual']

In [ ]:
#dropping the 2 outliers
train_clean = train_clean.drop([960,1885,125,348]).reset_index(drop = True)

In [ ]:
train_clean.shape

After cleaning, 4 entries have been removed from the dataset and we are left with 2047 rows for 81 features.

<a id='eda'></a>
# Exploratory Data Analysis

We will conduct EDA by grouping features into their respecive data types to allow for cross examination between features of similar type.

In [ ]:
train_clean.columns

In [ ]:
#let us categorize our target and features
target = 'saleprice'

cont_features = ['lot_frontage','lot_area','mas_vnr_area',
                 'bsmtfin_sf_1','bsmtfin_sf_2','bsmt_unf_sf',
                 'total_bsmt_sf','1st_flr_sf','2nd_flr_sf',
                 'low_qual_fin_sf','gr_liv_area','garage_area',
                 'wood_deck_sf','open_porch_sf','enclosed_porch',
                 '3ssn_porch','screen_porch','pool_area','misc_val']

disc_features = ['year_built','year_remod_add','bsmt_full_bath',
                 'bsmt_half_bath','full_bath','half_bath','bedroom_abvgr',
                 'kitchen_abvgr','totrms_abvgrd','fireplaces','garage_yr_blt',
                 'garage_cars','mo_sold','yr_sold']

nomi_features = ['ms_subclass','ms_zoning','street','alley',
                 'land_contour','lot_config','neighborhood',
                 'condition_1','condition_2','bldg_type','house_style',
                 'roof_style','roof_matl','exterior_1st','exterior_2nd',
                 'mas_vnr_type','foundation','heating','central_air',
                 'garage_type','misc_feature','sale_type']

#We take out id and pid for the EDA as they represent a unique identification number and have no predictive value.
ordi_features = [x for x in train_clean.columns if x not in ([target] + cont_features + disc_features + nomi_features + ['id','pid'])]

## Overview of correlations between saleprice and numerical features

The heatmap between all numerical features is plotted with the column in relation to saleprice in focus. This will help direct our focus to specific features that have a strong relationship to the saleprice.

In [ ]:
plt.figure(figsize=(10,25))
data = train_clean
sns.heatmap(data.corr()[['saleprice']].sort_values('saleprice',ascending=False), cmap = 'coolwarm', annot = True);

<a id='cont'></a>
## Continuous Features


In [ ]:
cont_features

### Continuous features selection

In [ ]:
#examine correlations between continuous features and target
plt.figure(figsize=(12,12))
data = train_clean[[target] + cont_features]
#mask = np.triu(data.corr())
sns.heatmap(data.corr()[['saleprice']].sort_values('saleprice',ascending=False), annot = True, cmap = 'coolwarm');

First we examine the correlations of the continuous features with the target. Those that exhibit strong correlations (>0.5) include:

- `gr_liv_area`
- `total_bsmt_sf`
- `garage_area`
- `1st_flr_sf`
- `mas_vnr_area`

Note that there are are many continuous features and of these features exhibit moderate correlations with one another, indicating collinearity.

Let's take a closer look at the features and see if we can reduce the noise/complexity

In [ ]:
train_clean[cont_features].head(10)

In [ ]:
#examine correlations between continuous features and target
plt.figure(figsize=(12,12))
data = train_clean[[target] + cont_features]
#mask = np.triu(data.corr())
sns.heatmap(data.corr()[['gr_liv_area']].sort_values('gr_liv_area',ascending=False), annot = True, cmap = 'coolwarm');

Some obvious relationships are: 

`gr_liv_area` = `1st_flr_sf` + `2nf_flr_sf` + `low_qual_fin_sf`

`total_bsmt_sf` = `bsmtfin_sf_1` + `bsmtfin_sf_2` + `bsmt_unf_sf`

Since `gr_liv_area` has a higher correlation to the saleprice than its components we will use `gr_liv_area` and drop the features on the RHS of the equation. The same applies to `total_bsmt_sf`.

This leaves us with:

- `gr_liv_area`
- `total_bsmt_sf`
- `garage_area`
- `mas_vnr_area`

In [ ]:
sns.pairplot(data = train_clean, x_vars = ['gr_liv_area', 'total_bsmt_sf', 'garage_area', 'mas_vnr_area'], y_vars = 'saleprice', kind = 'reg', height = 4, markers = 'x');

In [ ]:
cont_select = ['gr_liv_area', 'total_bsmt_sf', 'garage_area','mas_vnr_area']

<a id='disc'></a>
## Discrete Features


In [ ]:
disc_features

In [ ]:
#heatmap showing correlations between all features
plt.figure(figsize=(15,15))
data = train_clean[disc_features + ['saleprice']]
mask = np.triu(data.corr())
sns.heatmap(data.corr(), annot = True, cmap = 'coolwarm', mask = mask);

In [ ]:
plt.figure(figsize=(15,15))
data = train_clean[disc_features + ['saleprice']]
#mask = np.triu(data.corr())
sns.heatmap(data.corr()[['saleprice']].sort_values('saleprice',ascending=False), annot = True, cmap = 'coolwarm');

Features that show strong correlation to one another include:
- `garage_cars`
- `year_built`
- `year_remod_add`
- `garage_yr_blt`
- `full_bath`
- `totrms_abvgrd`

Several of these features can be engineered in improve their predictive value

### Feature Engineering

#### Property Age
`year_built` can be changed to age of property by subtracting the value from the year in which the study was conducted (2010)
`garage_yr_blt` can be engineered into age in  similar fashion

#### Years since last renovation
`year_remod_add` can be changed to the time since the property was last renovated by subtracting its value from 2010.

#### Number of baths
A new feature `tot_baths` can be engineered by adding the `full_bath` + `bsmt_full_bath` + 0.5*(`half_bath` + `bsmt_half_bath`)

#### Month sold
Currently, the `mo_sold` is not a useful feature as the months are repeated each year. A new nominal feature, `season`, can be engineered and perhaps may give more useful insight into how sale price varies with seasonal changes.

In [ ]:
engi_features = ['year_built','garage_yr_blt','year_remod_add','full_bath', 'bsmt_full_bath', 'half_bath', 'bsmt_half_bath','mo_sold']
features_to_engineer=train_clean[engi_features]

In [ ]:
features_to_engineer

In [ ]:
features_to_engineer['age'] = 2010 - features_to_engineer['year_built']
features_to_engineer['since_reno'] = 2010 - features_to_engineer['year_remod_add']
features_to_engineer['garage_age'] = 2010 - features_to_engineer['garage_yr_blt']
features_to_engineer['tot_baths'] = features_to_engineer['full_bath'] + features_to_engineer['bsmt_full_bath'] + 0.5*(features_to_engineer['half_bath'] + features_to_engineer['bsmt_half_bath'])

In [ ]:
seasons_dict = {'Spring': [2,3,4],
                'Summer' : [5,6,7],
                'Autumn' : [8,9,10],
                'Winter' : [11,12,1]}
features_to_engineer['season'] = features_to_engineer['mo_sold'].map(lambda x: [i for i in seasons_dict if x in seasons_dict[i]][0])
seasons = features_to_engineer['season']

In [ ]:
features_to_engineer

In [ ]:
#remove the original features
features_to_engineer.drop(columns = engi_features, inplace = True)

In [ ]:
features_to_engineer

Now combining the newly engineered features to the dataset to compare its relationship with saleprice.

In [ ]:
train_clean_1 = pd.concat([train_clean, features_to_engineer], axis = 1)

In [ ]:
plt.figure(figsize=(15,15))
data = train_clean_1[list(features_to_engineer.columns)+ disc_features + ['saleprice']]
mask = np.triu(data.corr())
sns.heatmap(data.corr(), annot = True, cmap = 'coolwarm', mask = mask);

### Discrete features selection

In [ ]:
plt.figure(figsize=(15,15))
data = train_clean_1[list(features_to_engineer.columns)+ disc_features + ['saleprice']]
#mask = np.triu(data.corr())
sns.heatmap(data.corr()[['saleprice']].sort_values('saleprice',ascending=False), annot = True, cmap = 'coolwarm');

We observe an improvement in correlation of `tot_baths` to saleprice as compared to its components. Also, the features which were converted to age did not change in magnitude but became negative. Therefore it is inconsequential to perform this particular manipulation. However since it makes more sense to use  `age` rather than the `year_built`, we will use the manipulated feature instead.

`age` shows a strong correlation (r = 0.85) to `garage_age` and is likely that the features are collinear. Most houses will be built around the same time as their garage, unless the garage does not exist.

I also decided to drop `totrms_abvgrd` and `garage_cars` as after several iterations of linear regression modeling, the lasso coefficient for the features alwas tended to 0. This shows that there is little or no linear relationship to saleprice, meaning that these features did not add any predictive value to the model.

I ended up not selecting any of the original discrete features. Instead I used the following engineered features.

Engineered features:

- `tot_baths`
- `age`
- `since_reno`


In [ ]:
disc_select = ['garage_cars', 'totrms_abvgrd']
engi_select = engi_features

<a id='ordi'></a>
## Ordinal Features


In [ ]:
ordi_features

Ordinal features are categorical features where the values are associated with a ranking. Apart from `overall_qual` and `overall_cond`, most of the ordinal features are string datatypes and must be encoded to a numerical value associated with their ranking. The dictionary below establishes the order and value associated with the different categories for each ordinal feature.

In [ ]:
#creating dictionary to map all ordinal variables, including values not present in training set. 
#A value of 0 is usually assigned to values associated with the typical/standard quality. 
#Otherwise 0 is assigned to having 'None' quality of the feature

ordinal_dict = {'lot_shape': {'Reg':0, 'IR1':1, 'IR2':2, 'IR3':3}, #mapping lot_shape. Ranking represents measure of irregularity
               
               'utilities': {'AllPub':0, 'NoSewr':1, 'NoSeWa':2, 'ELO':3}, #mapping utilities. Ranking is based on 0 = having all public utilities, 3 = minimum utilities(electricity only) indicating a reduction in quality of life.
               
               'land_slope': {'Gtl':0, 'Mod':1, 'Sev':2}, #mapping land_slope. Ranking is based on slope severity.
                
               'exter_qual': {'Po':-4, 'Fa':-2, 'TA':0, 'Gd':2, 'Ex':4}, #mapping exter_qual, kitchen_qual, exter_cond and heating_qc
               'kitchen_qual':{'Po':-4, 'Fa':-2, 'TA':0, 'Gd':2, 'Ex':4}, #these 4 variables take a similar ranking where 0 represents typical/average quality
               'exter_cond': {'Po':-4, 'Fa':-2, 'TA':0, 'Gd':2, 'Ex':4}, #-ve values indicate quality below average and +ve values indicate quality above average
               'heating_qc': {'Po':-4, 'Fa':-2, 'TA':0, 'Gd':2, 'Ex':4}, #magnitude is set as 2 to amplify effects of 'good' and 'fair' quality
                
               'pool_qc': {'None':0, 'Fa':1, 'TA':2, 'Gd':3, 'Ex':4}, #mapping pool_qc, bsmt_qual, bsmt_cond, fireplace_qu, garage_qual, garage_cond
               'bsmt_qual': {'None':0, 'Po':1,'Fa':2,'TA':3, 'Gd':4, 'Ex':5}, #these variables also take a similar ranking
               'bsmt_cond': {'None':0, 'Po':1,'Fa':2,'TA':3, 'Gd':4, 'Ex':5}, # 0 represents having no pool/basement/fireplace/garage
               'fireplace_qu': {'None':0, 'Po':1,'Fa':2,'TA':3, 'Gd':4, 'Ex':5}, #ranking is in increasing order of quality
               'garage_qual': {'None':0, 'Po':1,'Fa':2,'TA':3, 'Gd':4, 'Ex':5},
               'garage_cond': {'None':0, 'Po':1,'Fa':2,'TA':3, 'Gd':4, 'Ex':5},
                
               'bsmt_exposure': {'No':0, 'None':0, 'Mn':1, 'Av':2, 'Gd':3}, #mapping bsmt_exposure. Having no basement and no exposure are considered to be of the same value of 0.
                
               'bsmtfin_type_1': {'None':0, 'Unf':1,'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6}, #mapping bsmtfin_type_1 bsmtfin_type_2
               'bsmtfin_type_2': {'None':0, 'Unf':1,'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6}, #Ranking is based on quality of finish
                
               'electrical': {'SBrkr':0, 'FuseA':1, 'FuseF':2, 'FuseP':3, 'Mix':2.5}, #mapping electrical. Ranking is based on 0 having standard circuit breakers in in order of decreasing electrical system sophistication. 
                #Mixed electrical system is considered to be between FuseF and FuseP
                
               'functional':{'Typ':0,'Min1':1,'Min2':2, 'Mod':3, 'Maj1':4, 'Maj2':5, 'Sev':6, 'Sal':7}, #mapping functional. Ranking is based on 0 having typical home functionality and in decreasing order of functionality
                
               'garage_finish':{'None':0, 'Unf':1, 'RFn':2, 'Fin':3}, #mapping garage_finish. 0 represents having no garage and ranking is in order of quality of finish.
                
               'paved_drive':{'N':0, 'P':1, 'Y':2}, #mapping paved_drive. Ranking is in order of amount of paving on driveway.
                
               'fence':{'None':0, 'MnWw':1, 'GdWo':2, 'MnPrv':3, 'GdPrv':4} #mapping fence. 0 represents having no fence and ranking is in increasing levels of privacy
               }

In [ ]:
#mapping all ordinal features accordingly
for i in ordinal_dict:
    train_clean[i] = train_clean[i].map(ordinal_dict[i])

In [ ]:
train_clean[ordi_features].head()

### Ordinal Features Selection

In [ ]:
plt.figure(figsize=(15,15))
data = train_clean[ordi_features + ['saleprice']]
mask = np.triu(data.corr())
sns.heatmap(data.corr(), annot = True, cmap = 'coolwarm', mask = mask);

In [ ]:
#Zoom in on sale price
plt.figure(figsize=(15,15))
data = train_clean[ordi_features + ['saleprice']]
sns.heatmap(data.corr()[['saleprice']].sort_values('saleprice',ascending=False), annot = True, cmap = 'coolwarm');

We want to focus on the features with strongest correlation to saleprice (corr > 0.5). These are:

- `overall_qual`
- `exter_qual`
- `kitchen_qual`
- `bsmt_qual`
- `garage_finish`
- `fireplace_qu`

These features indicate the strongest linear relationship to the target variable. However we also note that `overall_qual` is strongly correlated to some of the above features. In the figure below we zoom in on the column for `overall_qual` from the heatmap.

In [ ]:
plt.figure(figsize=(15,15))
data = train_clean[ordi_features + ['saleprice']]
sns.heatmap(data.corr()[['overall_qual']].sort_values('overall_qual',ascending=False), annot = True, cmap = 'coolwarm');

Take note that the features which are strongly correlated to `overall_qual` in descending order are:

- `exter_qual`
- `kitchen_qual`
- `bsmt_qual`
- `garage_finish`

This makes sense as these indicators are subsets of the overall quality of the house. We can drop these features from consideration and just use `overall_qual` to avoid overfitting, and consider the dropped features as components which make up the overall quality rating. 

This leaves us with

- `overall_qual`
- `fireplace_qu`

In [ ]:
ordi_select = ['overall_qual','fireplace_qu']

<a id='nomi'></a>
## Nominal Features


In [ ]:
nomi_features

In [ ]:
def subplot_count(df, list_of_titles,size):
    nrows = int(np.ceil(len(list_of_titles)/2)) # Makes sure you have enough rows
    fig, ax = plt.subplots(nrows=nrows, ncols=2, figsize=size) # You'll want to specify your figsize
    ax = ax.ravel()
    for i, column_x in enumerate(df):                  # Gives us an index and columns
        sns.countplot(data=df,
                    x = column_x,
                    ax= ax[i])       # Plotting diagrams
        
        
    if len(list_of_titles) % 2 == 1:                             # Turn off odd number of subplots
        ax[-1].axis('off')
    sns.set(font_scale=1.5)    # Set scale for plots
    plt.tight_layout()         # Move the plots so they don't overlap

There are several nominal features with different categories for each. It might be overly complicated to inlude each dummy columns to represent each category per feature. To narrow down our selected features, we eliminate features that are heavily skewed to one category as it would not provide much predictive value. To visualize this, we can plot a countplot of each feature.

In [ ]:
subplot_count(df = train_clean[nomi_features],
           list_of_titles = nomi_features,
           size = (15,28))

Clearly there are many features that *heavily favour one category*. We can eliminate those by *setting a threshold of 75% proportion of the majority category*. This will eliminate features that have over 75% data entries for a single category.

In [ ]:
#function to select nominal features based on the spread of the data between its respective categories

def select_nominal(df, features, threshold):
    result = []
    for i , v in enumerate(features):
        count = df[v].value_counts()
        weight = count[0]/count.sum()
        if weight < threshold:
            result.append(v)
    return result

In [ ]:
choose_nomi = select_nominal(train_clean, nomi_features, threshold = 0.75)
nomi_filter = train_clean[choose_nomi]
nomi_filter = pd.concat([nomi_filter, seasons], axis = 1)
nomi_filter

Out of 23 features, 13 were eliminated in the process. This leaves us with 10 to explore further.

For the remaining features, we can visualize the effect of each category on the target using boxplots. For the feature to hold good predictive power, there must be a clear distinction in the range of saleprices covered by the boxplot. *There should be little or no overlap between the interquartile range of the plots*

In [ ]:
def subplot_box(df, list_of_titles,size):
    nrows = int(np.ceil((len(list_of_titles)-1)/2)) # Makes sure you have enough rows
    fig, ax = plt.subplots(nrows=nrows, ncols=2, figsize=size) # You'll want to specify your figsize
    ax = ax.ravel()
    for i, column_x in enumerate(df):# Gives us an index and columns
        
        if column_x != 'saleprice':
            by_med = df[[column_x,'saleprice']].groupby(column_x).median().sort_values(by = 'saleprice').index
            sns.boxplot(data=df,
                        x = 'saleprice',
                        y = column_x,
                        orient = 'h',
                        fliersize = 3,
                        order = by_med,
                        ax= ax[i])       # Plotting diagrams

In [ ]:
nomi_check = pd.concat([nomi_filter, train_clean['saleprice']], axis = 1)
subplot_box(nomi_check, 
           list(nomi_check.columns),
           size = (20,40))

In [ ]:
len(list(nomi_check.columns))

Some of the nominal features show a clear binary discintion between categories. Note that the engineered feature `season` was included in this analysis and there seems to be no clear distinction between categories for that feature. We drop the feature from the model.

In [ ]:
nomi_check[['neighborhood','saleprice']].groupby('neighborhood').describe()

In [ ]:
np.log(train_clean['saleprice']).mean()

In [ ]:
np.log(train_clean['saleprice']).std()

In [ ]:
log_sale = train_clean[['neighborhood','saleprice']]
log_sale['log_saleprice'] = np.log(log_sale['saleprice'])

In [ ]:
log_sale

In [ ]:
log_sale[['neighborhood','log_saleprice']].groupby('neighborhood').median().sort_values(by = 'log_saleprice')

We divide `neighborhood` into 3 categories distinguished by their median sale prices. Note that saleprice does not follow a normal distribution, and a log transformation is applied to the saleprice before calculating the mean and standard deviation of the log saleprice. The neighborhoods are classified based on whether their median log sale price is within 1 standard deviation away from the global mean log sale price. Since 1 standard deviation from the mean represents 34.1% of all the data, removing the middle portion of the data results in a more or less "equal" split of the data with approximately 33% belonging to the lower and upper class neighborhoods and the remaining 34% in the middle class neighborhoods.

In [ ]:
np.log(train_clean['saleprice']).mean() + np.log(train_clean['saleprice']).std()

In [ ]:
np.log(train_clean['saleprice']).mean() - np.log(train_clean['saleprice']).std()

We can see a clear split between neighborhoods. The neighborhoods belonging to the 'upper class' will be represented by a new dummy variable `neighborhood_h` and includes:
- `StoneBr`
- `NridgHt`
- `NoRidge`
- `GrnHill`
- `Veenker`

For the lower classes, the dummmy variable created is `neighborhood_l` and it encompasses:
- `BrDale`
- `IDOTRR`
- `MeadowV`


All other remaining neighborhoods belong in the middle class therefore there isn't a need to create a third dummy column.

In [ ]:
nomi_check[['garage_type','saleprice']].groupby('garage_type').describe()

There are 2 groups of `garage_type`.  `Attchd` and `Builtin` for `garage_type` tends to fetch a higher saleprice than the other garage types. We can create a new feature to represent the group together `Attchd` and `Builtin`.

In [ ]:
nomi_select = ['neighborhood','garage_type']
engi_features_2 = nomi_check[nomi_select]
engi_features_2

In [ ]:

engi_features_2['neighbor_h'] = engi_features_2['neighborhood'].map(lambda x: 1 if x == 'StoneBr' or x =='NridgHt' or x == 'NoRidge' or x == 'GrnHill' or x == 'Veenker' else 0)
engi_features_2['neighbor_l'] = engi_features_2['neighborhood'].map(lambda x: 1 if x == 'BrDale' or x == 'IDOTRR' or x == 'MeadowV' else 0)


engi_features_2['garage_type_a'] = engi_features_2['garage_type'].map(lambda x: 1 if x == 'Attchd' or x == 'BuiltIn' else 0)
engi_features_2

In [ ]:
engi_features_2.drop(columns = ['neighborhood','garage_type'], inplace = True)
engi_features_2

In [ ]:
data = pd.concat([engi_features_2,train_clean['saleprice']], axis = 1)

In [ ]:
sns.scatterplot(y = data['neighbor_h'], x = data['saleprice']);

In [ ]:
sns.scatterplot(y = data['neighbor_l'], x = data['saleprice']);

In [ ]:
sns.scatterplot(y = data['garage_type_a'], x = data['saleprice']);

There looks to be slight distinction between the groups for the new binary variables. More noticably in `neighbor_h` and `neighbor_l`.

<a id='poly'></a>
## Polynomial Features


To minimize the complexity with computing polynomial features over the entire dataset, we instead perform the analysis on the selected features.

In [ ]:
model_features = cont_select + disc_select + ordi_select + nomi_select + engi_select

In [ ]:
model_features

In [ ]:
model_df = train_clean[model_features]


#Discrete feature engineering
model_df['age'] = 2010 - model_df['year_built']
model_df['since_reno'] = 2010 - model_df['year_remod_add']
model_df['tot_baths'] = model_df['full_bath'] + model_df['bsmt_full_bath'] + 0.5*(model_df['half_bath'] + model_df['bsmt_half_bath'])

#Drop the original columns
model_df.drop(columns = engi_select, inplace = True)

#Ordinal features already mapped for train set

#Nominal features mapping
model_df['neighbor_h'] = model_df['neighborhood'].map(lambda x: 1 if x == 'StoneBr' or x =='NridgHt' or x == 'NoRidge' or x == 'GrnHill' else 0)
model_df['neighbor_l'] = model_df['neighborhood'].map(lambda x: 1 if x == 'Sawyer' or x == 'Names' or x == 'Edwards' or x == 'OldTown'
                                                                    or x == 'BrDale' or x == 'IDOTRR' or x == 'MeadowV' or x == 'BrkSide' 
                                                                    or x == 'NPkVill' or x == 'Blueste' or x == 'SWISU' else 0)
model_df['garage_type_a'] = model_df['garage_type'].map(lambda x: 1 if x == 'Attchd' or x == 'BuiltIn' else 0)

#Drop the original columns
model_df.drop(columns = nomi_select, inplace = True)

#Display the resulting model features
model_df

In [ ]:
poly = PolynomialFeatures(include_bias=False)
X_pol = poly.fit_transform(model_df)
X_pol.shape

In [ ]:
X_pol = pd.DataFrame(X_pol,columns=poly.get_feature_names(model_df.columns))

#create list of correlations between features with price
X_pol_corrs = X_pol.corrwith(train_clean[target])

#Shows features with highest positive correlation to price
X_pol_corrs.sort_values(ascending=False).head(20)

In [ ]:
X_pol['saleprice'] = train_clean['saleprice']

In [ ]:
sns.pairplot(data = X_pol, y_vars = 'saleprice', x_vars = ['gr_liv_area','gr_liv_area overall_qual','overall_qual'], height = 5, kind = 'reg');

In [ ]:
sns.pairplot(data = X_pol, y_vars = 'saleprice', x_vars = ['total_bsmt_sf','total_bsmt_sf overall_qual','overall_qual'], height = 5, kind = 'reg');

In [ ]:
sns.pairplot(data = X_pol, y_vars = 'saleprice', x_vars = ['garage_area','garage_area overall_qual','overall_qual'], height = 5, kind = 'reg');

We can see that the polynomial features can improve the predictive power of our original features. Some of the polynomial features show higher correlation with the sale price than the original features. From the above scatter plots, we can see that the polynomial features help to centre the datapoints closer to the regression line. However most of these polynomial features are collinear. After several model iterations using different combinations of polynomial features, I ended up using `gr_liv_area overall_qual` and `garage_area overall_qual`.

In [ ]:
model_df['gr_liv_area overall_qual'] = X_pol['gr_liv_area overall_qual']
model_df['garage_area overall_qual'] = X_pol['garage_area overall_qual']

#drop the unnecessary disc features
model_df.drop(columns =['garage_cars','totrms_abvgrd'], inplace = True)

In [ ]:
model_df

<a id='target'></a>
## Target Feature


The target feature, saleprice is continuous and we would expect it to follow a normal distribution over a large sample size. Plotting the distribution of the saleprice from the training dataset, we can see evidence of positive skewness. This means that the data is concentrated on the lower range of saleprices. The median and mode of the data is less than its mean.

In [ ]:
plt.figure(figsize = (10,10))
sns.histplot(data = train_clean,
            x = 'saleprice');

Skewness in the target variable can lead to inaccurate predictions such as underestimating saleprices. We can perform feature engineering on the target feature to normalize its distribution and reduce skewness. A common technique is to perform **logarithmic transformation of the target feature**.

In [ ]:
y_log = np.log(train_clean['saleprice'])
plt.figure(figsize = (10,10))
sns.histplot(y_log);

This is the final transformation to apply to the train dataset. After making predictions, the values of **predicted saleprice need to be exponentiated** to convert them back to their actual sale prices.

In [ ]:
model_df['saleprice'] = train_clean['saleprice']

<a id='select'></a>
## Features selected for model

In [ ]:
model_df.info()

There are a total of 14 features selected for our model. We will export this dataframe to be used in our modelling process.

In [ ]:
model_df.to_csv('./datasets/train_model_features.csv', index = False)

#save a copy of our original cleaned data
train_clean.to_csv('./datasets/train_clean.csv', index = False)